# Toy Correlation Tutorial

In this notebook we explore how different correlation assumptions impact a simple two-measurement combination. Both measurements provide an estimate of the same quantity with central values +1 and -1. Their statistical uncertainties are $\sqrt{2}$ and we introduce a single systematic source also of magnitude $\sqrt{2}$. All error-on-error terms ($\epsilon$) are set to zero so only the correlations influence the result.


In [ ]:
import os, sys

script_dir = os.getcwd()
gvm_root = os.path.abspath(os.path.join(script_dir, "../../"))
if gvm_root not in sys.path:
    sys.path.insert(0, gvm_root)

from gvm_toolkit import GVMCombination

The following cells build a `GVMCombination` for each correlation scenario. After running the fit we print the estimated mean (`mu_hat`), the 68% confidence interval and the goodness-of-fit (chi-square) value.


## 1. Decorrelated case
Here the systematic uncertainty is defined as independent for each measurement (no correlation matrix).

In [ ]:
comb = GVMCombination('correlations/decorrelated.yaml')
mu_hat = comb.fit_results['mu']
ci_low, ci_high, _ = comb.confidence_interval()
chi2 = comb.goodness_of_fit()
print(f'decorrelated: mu_hat={mu_hat:.4f}, CI=({ci_low:.4f}, {ci_high:.4f}), chi2={chi2:.3f}')


## 2. Correlation examples
We now compare three different correlation matrices for the systematic uncertainty:
1. **Diagonal**: off-diagonal terms are zero so the systematic acts independently.
2. **Fully correlated**: all coefficients are one so the systematic behaves as one shared nuisance parameter.
3. **Hybrid**: the off-diagonal coefficient is 0.5 representing a partial correlation.


In [ ]:
comb = GVMCombination('correlations/diag_corr.yaml')
mu_hat = comb.fit_results['mu']
ci_low, ci_high, _ = comb.confidence_interval()
chi2 = comb.goodness_of_fit()
print(f'diag_corr: mu_hat={mu_hat:.4f}, CI=({ci_low:.4f}, {ci_high:.4f}), chi2={chi2:.3f}')

comb = GVMCombination('correlations/full_corr.yaml')
mu_hat = comb.fit_results['mu']
ci_low, ci_high, _ = comb.confidence_interval()
chi2 = comb.goodness_of_fit()
print(f'full_corr: mu_hat={mu_hat:.4f}, CI=({ci_low:.4f}, {ci_high:.4f}), chi2={chi2:.3f}')

comb = GVMCombination('correlations/hybrid_corr.yaml')
mu_hat = comb.fit_results['mu']
ci_low, ci_high, _ = comb.confidence_interval()
chi2 = comb.goodness_of_fit()
print(f'hybrid_corr: mu_hat={mu_hat:.4f}, CI=({ci_low:.4f}, {ci_high:.4f}), chi2={chi2:.3f}')


## 3. Non-diagonal statistical covariance
The same systematic is fully correlated as above, but the statistical uncertainties are supplied via a covariance matrix with non-zero off-diagonal terms.

In [ ]:
comb = GVMCombination('correlations/stat_cov.yaml')
mu_hat = comb.fit_results['mu']
ci_low, ci_high, _ = comb.confidence_interval()
chi2 = comb.goodness_of_fit()
print(f'stat_cov: mu_hat={mu_hat:.4f}, CI=({ci_low:.4f}, {ci_high:.4f}), chi2={chi2:.3f}')


## Tip: modifying the combination
You can obtain the current input with `input_data()` and modify the returned dictionary. 
After editing, pass it to `update_data()` to apply the changes before refitting.


In [ ]:
comb = GVMCombination('correlations/decorrelated.yaml')

# extract current configuration
info = comb.input_data()

# change measurement and systematic values
info['data']['measurements']['m1']['central'] = 2.0
info['syst']['sys1']['values']['m1'] = 0.5

# update the systematic to be dependent with a new correlation matrix
info['syst']['sys1']['type'] = 'dependent'
info['syst']['sys1']['corr'] = np.array([[1.0, 0.3], [0.3, 1.0]])

# apply modifications and refit
comb.update_data(info)
comb.fit_results = comb.minimize()
print(f"updated mu_hat={comb.fit_results['mu']:.4f}")
